In [1]:
# King County, WA Housing Data: A Multicollinearity Journey

import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import pandas  as pd
import numpy   as np

In [2]:
housing = pd.read_csv("kc_house_data.csv", delimiter=',')
housing.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
# Drop most of the columns for this exercise in linear regression.
housing.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [4]:
housing['logprice'] = np.log(housing['price'])
housing = housing[['logprice', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'sqft_above', 'sqft_basement', 'lat', 'long']]

In [5]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 9 columns):
logprice         21613 non-null float64
bedrooms         21613 non-null int64
bathrooms        21613 non-null float64
sqft_living      21613 non-null int64
sqft_lot         21613 non-null int64
sqft_above       21613 non-null int64
sqft_basement    21613 non-null int64
lat              21613 non-null float64
long             21613 non-null float64
dtypes: float64(4), int64(5)
memory usage: 1.5 MB


In [6]:
# If the number of rows where this inequality holds is 0, then we have perfect multicollinearity.
sum(housing['sqft_living'] != housing['sqft_above'] + housing['sqft_basement'])

0

In [7]:
# Now let's examine how Statsmodel treats this fact of multicollinearity.
# Spoiler alert: it doesn't care.
# No feature engineering here besides log price. We're not trying to do the OLS 
# "nicely", just showing what happens when you do it at all.

simple_regression = ['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'sqft_above', 'sqft_basement', 'lat', 'long']
outcome = 'logprice'
pred_sum = '+'.join(simple_regression)
formula = outcome + "~" + pred_sum

model = ols(formula=formula, data=housing).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               logprice   R-squared:                       0.666
Model:                            OLS   Adj. R-squared:                  0.666
Method:                 Least Squares   F-statistic:                     6158.
Date:                Mon, 02 Sep 2019   Prob (F-statistic):               0.00
Time:                        12:33:44   Log-Likelihood:                -4955.0
No. Observations:               21613   AIC:                             9926.
Df Residuals:                   21605   BIC:                             9990.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       -98.7028      2.032    -48.564      0.000    -102.687     -94.719
bedrooms         -0.0387      0.003    -13.942      0.000      -0.044      -0.033
bathrooms         0.0647      0.004     15.487      0.000       0.056       0.073
sqft_living       0.0002   2.65e-06     91.202      0.000       0.000       0.000
sqft_lot       3.986e-07   5.22e-08      7.635      0.000    2.96e-07    5.01e-07
sqft_above        0.0001   2.59e-06     55.777      0.000       0.000       0.000
sqft_basement  9.747e-05   3.42e-06     28.480      0.000    9.08e-05       0.000
lat               1.5463      0.015    101.540      0.000       1.516       1.576
long             -0.3061      0.016    -18.870      0.000      -0.338      -0.274
==============================================================================
Omnibus:                      593.070   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1414.126
Skew:                           0.097   Prob(JB):                    8.44e-308
Kurtosis:                       4.238   Cond. No.                     2.03e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.02e-21. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [8]:
simpler_regression = simple_regression.copy()
simpler_regression.remove('sqft_basement')
pred_sum = '+'.join(simpler_regression)
formula = outcome + "~" + pred_sum

model2 = ols(formula=formula, data=housing).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               logprice   R-squared:                       0.666
Model:                            OLS   Adj. R-squared:                  0.666
Method:                 Least Squares   F-statistic:                     6158.
Date:                Mon, 02 Sep 2019   Prob (F-statistic):               0.00
Time:                        12:33:45   Log-Likelihood:                -4955.0
No. Observations:               21613   AIC:                             9926.
Df Residuals:                   21605   BIC:                             9990.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     -98.7028      2.032    -48.564      0.000    -102.687     -94.719
bedrooms       -0.0387      0.003    -13.942      0.000      -0.044      -0.033
bathrooms       0.0647      0.004     15.487      0.000       0.056       0.073
sqft_living     0.0003   5.55e-06     61.148      0.000       0.000       0.000
sqft_lot     3.986e-07   5.22e-08      7.635      0.000    2.96e-07    5.01e-07
sqft_above   4.689e-05   5.46e-06      8.591      0.000    3.62e-05    5.76e-05
lat             1.5463      0.015    101.540      0.000       1.516       1.576
long           -0.3061      0.016    -18.870      0.000      -0.338      -0.274
==============================================================================
Omnibus:                      593.070   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1414.126
Skew:                           0.097   Prob(JB):                    8.44e-308
Kurtosis:                       4.238   Cond. No.                     4.33e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.33e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
housing_array = np.matrix(housing[['sqft_living', 'sqft_lot', 
                                   'sqft_above', 'sqft_basement']])

In [10]:
housing_array # perfect multicollinearity: col1 = col3 + col4

matrix([[ 1180,  5650,  1180,     0],
        [ 2570,  7242,  2170,   400],
        [  770, 10000,   770,     0],
        ...,
        [ 1020,  1350,  1020,     0],
        [ 1600,  2388,  1600,     0],
        [ 1020,  1076,  1020,     0]])

In [11]:
multicol = housing_array[:,0] - housing_array[:,2] - housing_array[:,3]
multicol.sum() # verifying that perfect multicollinearity is present

0

In [12]:
xtx = housing_array.T * housing_array

In [13]:
np.linalg.det(xtx) # practically infinite, should not be invertible

4.47544781102605e+28

In [14]:
np.linalg.inv(xtx) # row 1 is approx -row 3 and -row 4, 
# it's col 2 that's the problem, 12 places in

matrix([[ 3.13510075e+05, -3.67261024e-12, -3.13510075e+05,
         -3.13510075e+05],
        [-4.77529661e-13,  2.79154173e-14,  2.71585715e-13,
          4.50351186e-13],
        [-3.13510075e+05,  3.43948436e-12,  3.13510075e+05,
          3.13510075e+05],
        [-3.13510075e+05,  3.62397658e-12,  3.13510075e+05,
          3.13510075e+05]])